In [ ]:
#PART 1: Import libraries
import numpy as np
import networkx as nx
import pandas as pd
import random
from epyt import epanet
from geneticalgorithm2 import geneticalgorithm2 as ga
from geneticalgorithm2 import Generation, AlgorithmParams # classes for comfortable parameters setting and getting
from geneticalgorithm2 import Crossover, Mutations, Selection # classes for specific mutation and crossover behavior
from geneticalgorithm2 import Population_initializer # for creating better start population
from geneticalgorithm2 import np_lru_cache # for cache function (if u want)
from geneticalgorithm2 import plot_pop_scores # for plotting population scores, if u want
from geneticalgorithm2 import Callbacks # simple callbacks (will be deprecated)
from geneticalgorithm2 import Actions, ActionConditions, MiddleCallbacks # middle callbacks

#PART 2: Defining the initial network dimensions m*n
G = nx.grid_2d_graph(5,4) #grid graph dimention for example 7*6

#PART 3: Applying a random process to the initial network and assigning random values of demand, Hazen coefficient, and length to nodes and edges
key_node_initial,random_node_initial=[],[] #define key_node_initial,random_node_initial for grid_graph dictionary
for i in G:
    random_node_initial.append(np.random.randint(0,101))    #random_node_initial demand for nodes Within range(0,101)
    key_node_initial.append(i)
zip_keyrandom=(zip(key_node_initial,random_node_initial))
rndnode_dict=dict(zip_keyrandom)
nx.set_node_attributes(G,rndnode_dict,'random demand')
df_demand=pd.DataFrame.from_dict(dict(G.nodes(data=True)),orient='index')
df_demand_org=df_demand.copy()

#PART 3-1:nodes with random demands=0 are removed from both the graph geometry and the database to create a random graph 
df_demand=df_demand[df_demand['random demand'] > 0][ : ] 
get_attrib=nx.get_node_attributes(G,'random demand')     
for i in get_attrib:
    if get_attrib[i]<1:
        remove_zero_node=G.remove_node(i)
neighbours = {n: len(list(nx.all_neighbors(G, n))) for n in G.nodes}
for i in neighbours:
    if neighbours[i]<1:
        k=G.remove_node(i)
final_get_attrib=nx.get_node_attributes(G,'random demand')    
random_node=list(final_get_attrib.values())
reservoir_elev=np.random.randint(20,90)    

#PART 3-2:Assigning random values of the Hazen coefficient(80,121) and length(20,101) to edges
dimeter_edges,randomlength_edge,randomhazen_edge,G_edges=[],[],[],[]
for i in G.edges():
    G_edges.append(i)
    r_e2=np.random.randint(20,101)   #random integer length for edges
    randomlength_edge.append(r_e2)   
    r_e3=np.random.randint(80,121)    #random integer hazen for edges
    randomhazen_edge.append(r_e3)
    dimeter_edges.append('D'+str(i[0][0])+str(i[0][0])+str(i[0][1])+str(i[1][0])+str(i[1][1]))
randomlength_edge.append(np.random.randint(20,101))
randomhazen_edge.append(np.random.randint(80,121))
dimeter_edges.append('DNode1R1') #Creating a connection index of the Reservoir node to the network graph
num_hyper=len(dimeter_edges) # Number of Hyperarameters
print('diameter of the pipes, or number of Hyperparameters',num_hyper)
print('unknown diameter',dimeter_edges)# This refers to the diameter of the pipes, or Hyperparameters.
print('nodes demand',random_node)
print('pipe length',randomlength_edge)
print('pipe hazen',randomhazen_edge)
if nx.is_connected(G)==True:
    print('//////////////CONNECTED///////////')
else:
    print('//////////////UNCONNECTED/////////')
    
#PART 4:Epanet simulator model function to control the results of the optimizer algorithm
def hydrauic_sim(x):
    testinp = 'TEST.inp'
    d = epanet(testinp, 'CREATE')
    d.initializeEPANET(d.ToolkitConstants.EN_LPS, d.ToolkitConstants.EN_HW)

    #PART 4-1: Adding node attributes to the water supply network graph in the simulator
    m=0
    for i in G:
        index1 = d.addNodeJunction(str(i[0])+str(i[0])+str(i[1]))
        d.setNodeJunctionData(index1, 0, random_node[m], '')
        d.setNodeCoordinates(index1,[i[0],i[1]])
        m+=1
        if m==len(G):
            connect_res=str(i[0])+str(i[0])+str(i[1])
    
    #PART 4-2: Adding edge attributes to the water supply network graph in the simulator      
    n=0
    for i in G.edges():
        index2 = d.addLinkPipe('D'+str(i[0][0])+str(i[0][0])+str(i[0][1])+str(i[1][0])+str(i[1][1]),
                                  str(i[0][0])+str(i[0][0])+str(i[0][1]),
                                 str(i[1][0])+str(i[1][0])+str(i[1][1]))
        d.setLinkPipeData(index2,  randomlength_edge[n], x[n], randomhazen_edge[n], 0)
        n+=1
        
    #PART 4-3: Adding Reservoir attributes to the water supply network graph in the simulator      
    index3 = d.addNodeReservoir('R1')
    d.setNodeElevations(index3, reservoir_elev)
    d.setNodeCoordinates(index3, [-1, -1])   
    index4 = d.addLinkPipe('DNode1R1',connect_res, 'R1')
    d.setLinkPipeData(index4, randomlength_edge[-1] , x[-1],randomhazen_edge[-1], 0) 
    
    #PART 4-4:Complete analysis and implementation of the hydraulic model
    d.openHydraulicAnalysis()
    d.initializeHydraulicAnalysis()
    d.runHydraulicAnalysis()
        
    P,D,V,F,Diameter,Length,Hazen=[],[],[],[],[],[],[]
    P.append(d.getNodePressure())
    D.append(d.getNodeActualDemand())
    V.append(d.getLinkVelocity())
    F.append(d.getLinkFlows())
    Diameter.append(d.getLinkDiameter())
    Length.append(d.getLinkLength())
    Hazen.append(d.getLinkRoughnessCoeff())
        
    d.closeHydraulicAnalysis()
    d.saveInputFile(testinp)
    d.deleteProject()
    return P
#PART 5:Function in order to decoding results of optimization algorithm to commercial diameters
def convert_commercial(orginal):
    commercial_index={'1':16,'2':20,'3':25,'4':32,'5':40,'6':50,'7':63,'8':75,'9':90,'10':110,'11':125,
            '12':160,'13':200,'14':250,'15':315,'16':400,'17':500,'18':600,'19':800,'20':1000}
    decode=np.array([round(1+(19*i)/1000) for i in orginal])
    input_model=[]
    for i in decode:
        input_model.append(commercial_index[str(i)])
    input_model=np.array(input_model)
    return input_model
cost_commercial={'16':10.34,'20':11.18,'25':12.22,'32':13.69,'40':15.36,'50':17.45,'63':20.17,
                 '75':22.67,'90':25.81,'110':30.89,'125':35.38,'160':48.84,'200':66.80,'250':95.25,
                 '315':141.83,'400':216.60,'500':327.50,'600':438.40,'800':660.20,'1000':882.00}

#PART 6:Definition and implementation of a single-objective genetic algorithm optimization model with pressure constraint control at nodes
def f(X):    
    input_simulator=convert_commercial(X)    
    sim_out=hydrauic_sim(input_simulator)
    net_sim_out=sim_out[0][:-1]   
    reject,pen,vio,sum_obj=0,0,0,0
    for i in net_sim_out:
        if i<15:  #minimum pressure head at demand nodes is 15 meters
            reject+=1
            vio+=15-i
    if reject>0:
        pen=80*vio   
    for i in range(num_hyper):
        sum_obj+=np.array(randomlength_edge[i])*(cost_commercial[str(input_simulator[i])]) #cost function
    return sum_obj*(1+pen)
varbound = [[0,1000]]*num_hyper
model = ga(f, dimension = num_hyper, 
                variable_type='int', 
                variable_boundaries = varbound,
                algorithm_parameters=AlgorithmParams(
                     max_num_iteration =250,
                     population_size = num_hyper*12,
                     mutation_probability = 0.05,
                     mutation_discrete_probability = None,
                     elit_ratio = 0.01,
                     parents_portion = 0.3,
                     crossover_type ='uniform',
                     mutation_type = 'uniform_by_center',
                     mutation_discrete_type = 'uniform_discrete',
                     selection_type = 'roulette',
                     max_iteration_without_improv = None
                     )
            )
result=model.run()
print(result.variable)
print(result.score)

#PART 7:Viewing the results of the GA output diameters and accept or reject the results according to the pressure constraint. 
output_ga=result.variable
Design_Diameter=convert_commercial(output_ga)
print('Design_Diameter',Design_Diameter)
simout_Pressure=hydrauic_sim(Design_Diameter)
print('simout_Pressure',simout_Pressure)
negative_pressure=0
for i in range(len(simout_Pressure[0])):
    if simout_Pressure[0][i]<0:
        negative_pressure+=1
if negative_pressure>0:
    print('------------reject----------')    
else:
    print('------------accept----------')
